In [1]:
import ee
import geopandas as gpd
import pandas as pd
import pandas as pd
import json

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
data = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE");

In [ ]:
dataset = data.filterDate('2018-5-1', '2020-5-30')

In [ ]:
total = dataset.reduce(ee.Reducer.sum());

In [ ]:
ranchi = gpd.read_file("./ranchi/ranchi_new.shp")

In [ ]:
js = json.loads(ranchi.to_json())
shpz = ee.Geometry(ee.FeatureCollection(js).geometry())

In [ ]:
stats = total.reduceRegion(
  reducer= ee.Reducer.mean(),
  geometry= shpz,
  scale= 5000,
  )

In [ ]:
print("Amount = ",stats.get('pr_sum').getInfo())

In [ ]:
count = total.reduceRegion(
  reducer= ee.Reducer.count(),
  geometry= shpz,
  scale= 5000,
  )

In [ ]:
print("Total Pixel = ", count.get('pr_sum').getInfo())

In [ ]:
totalRanchi = total.clip(shpz)
vectors = totalRanchi.sampleRegions(
  collection=shpz,
  geometries= True,
  scale= 5000,
)
vec =(vectors.getInfo())
print(type(vec))
print(vec)

In [ ]:
pd.json_normalize(vec['properties'])

In [ ]:
m = pd.json_normalize(vec)

In [ ]:
df2 = pd.DataFrame(m)

In [ ]:
print(df2)

In [ ]:
resultz = json.dumps(vec, indent = 3, sort_keys = True)
rio = resultz
lenen = rio.split("\n", 2)[2]
lenen_back = '\n'.join(lenen.split('\n')[:-21])
print(lenen_back)